In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.__version__)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
11.7
2.0.1+cu117
1
NVIDIA RTX A4500


In [3]:
from alike import ALike, configs
import cv2
import numpy as np
import torch.nn.functional as F

In [4]:
def computes_alike_keypoints(img, model_nm="alike-n", device="cuda", top_k=-1, scores_th=0.2, n_limit=5000):

    img_permuted = img.permute(2, 1, 0)
    model = ALike(**configs[model_nm],
        device=device,
        top_k=top_k,
        scores_th=scores_th,
        n_limit=n_limit)
    img_rgb = cv2.cvtColor(img_permuted.numpy(), cv2.COLOR_BGR2RGB)
    pred = model(img_rgb)
    kpts = pred["keypoints"]
    desc = pred["descriptors"]
    scores = pred["scores"]
    score_map = pred["scores_map"]
    print(kpts.shape, desc.shape, scores.shape, score_map.shape)

    kpt_details = np.zeros((kpts.shape[0],4))

    kpt_details[:,0] = kpts[:,1]
    kpt_details[:,1] = kpts[:,0]
    kpt_details[:,2] = scores.squeeze()
    kpt_details[:,3] = scores.squeeze()

    if len(kpts)>0:
        return torch.from_numpy(kpts), torch.from_numpy(desc)
    return None